#### Day 16 - A
Find the optimal path from the start point (S) to the end point (E).

Taking a step has a cost of 1.
Taking a step has a cost of 1000.

In [9]:
#Import Libraries and settings
import bisect
from copy import deepcopy

possible_optimals = []

settings = {
    "day": "16",
    "test_data": 0
}

In [10]:
#Load Input
def load_input(settings, initial_dir=">"):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    grid = []

    for idx_y, line in enumerate(lines):
        if "S" in line:
            idx_x = line.find("S")
            starting_loc = (idx_x, idx_y)
            grid.append(list(line[:idx_x] + initial_dir + line[idx_x+1:]))
        else:
            grid.append(list(line))

    return grid, starting_loc

GRID_BASE, starting_loc = load_input(settings)
GRID_WIDTH = len(GRID_BASE[0])
GRID_HEIGHT = len(GRID_BASE)

In [11]:
#Convert compass direction into a dir tuple
def translate_step(dir_g, mag=1):
    v = 0
    h = 0

    if "^" in dir_g:
        v = -mag
    elif "v" in dir_g:
        v = mag

    if ">" in dir_g:
        h = mag
    elif "<" in dir_g:
        h = -mag

    return (h, v)

#Apply a direction to a location
def apply_move(loc, dir, mag=1):
    step = translate_step(dir, mag)
    return (loc[0]+step[0], loc[1]+step[1])

In [12]:
#Function to inverse a direction
def dir_inv(dir):
    dir_inv_dict = {
        "^":"v",
        "v":"^",
        ">":"<",
        "<":">"
    }
    return dir_inv_dict[dir]

#Get all valid neighbours for a given state in the grid
def get_neighbours(grid, prev_move, history):

    loc = prev_move[0][0]
    dir = prev_move[0][1]
    base_cost = prev_move[1]

    #Get possible turns
    if dir == "^" or dir == "v":
        pos_dirs = [dir, "<", ">"]
    else:
        pos_dirs = [dir, "^", "v"]

    #Check which are already visited locations
    valid_neighbours = []
    for pos_dir in pos_dirs:

        next_loc = apply_move(loc, pos_dir)
        next_char = grid[next_loc[1]][next_loc[0]]

        #If taking a step
        if pos_dir == dir:
            move_cost = base_cost + 1
            #Check if end space
            if next_char == "E":
                next_dir = "E"
            else:
                next_dir = pos_dir
            move = (next_loc, next_dir)

        #If making a turn
        else:
            move_cost = base_cost + 1000
            move = (loc, pos_dir)
        
        #Check if direction free
        if next_char == "E":
            valid_neighbours.append((move, move_cost))
        elif next_char == ".":
            #Check the opposite move is not in history
            if (move[0], dir_inv(move[1])) not in history:
                #If in history then evaluate cost
                if move in history:
                    existing_cost = history[move]
                    if move_cost <= existing_cost:
                        valid_neighbours.append((move, move_cost))
                #Otherwise add the move
                else:
                    valid_neighbours.append((move, move_cost))

    return valid_neighbours

#Consider all moves through the grid until the end is found
def grid_walker(grid, loc, dir=">"):

    #Maintain history of previous moves
    history = {
        (loc, dir):0
    }

    #Initial move
    queue = get_neighbours(grid, ((loc, dir), 0), history)

    while queue:

        #Process move
        move, cost = queue[0]
        queue.pop(0)

        move_loc = move[0]
        move_dir = move[1]

        #Add to history
        if move not in history:
            history[move] = cost

        #Check if end condition found
        if move_dir == "E":
            return move_loc, cost, history

        #Get new items for the queue
        new_items = get_neighbours(grid, (move, cost), history)

        for ni in new_items:
                bisect.insort(queue, ni, key=lambda x:x[1])

In [40]:
end_loc, cost, history = grid_walker(GRID_BASE, starting_loc)
#Solution for part A
print(cost)

115500
